<a href="https://colab.research.google.com/github/AW17321/NPS_EAG_BETC/blob/main/CourseCatalogCrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#you only need to run this once per device or if something gets reset
!pip install -q scrapy
!pip install googlesearch-python
!pip install requests beautifulsoup4 lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.8/259.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 5.9 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "https://catalog.calpoly.edu/#/"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

# Extract only PDF
pdf_links = []
for a_tag in soup.find_all("a", href=True):
    href = a_tag["href"]
    if href.lower().endswith(".pdf"):
        full_url = urljoin(url, href)
        pdf_links.append(full_url)

for link in pdf_links:
    print(link)


https://catalog.calpoly.edu/homepage.pdf
https://catalog.calpoly.edu/pdf/2022-2026.pdf


In [ ]:
from googlesearch import search
import csv

#Make your google search better with https://ahrefs.com/blog/google-advanced-search-operators/
with open('test.csv', newline='') as csvfile:
    reader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for line in reader:
      #print(line[0]) #test to see if it read csv
      school = "site:" + line[0]
      keywords = " (battery OR energy storage)"
      searchMe = school + keywords
      print(searchMe)#check to see if we are searching correctly


      for result in search(searchMe, advanced=True):
        print(result) # check to see what I am outputting
        with open("output.csv", mode="a", newline="") as file:
          writer = csv.writer(file)
          file.write(str(result) + '\n')

site:https://www.auburn.edu/ (battery OR energy storage)
SearchResult(url=https://www.auburn.edu/cosam/news/articles/2024/11/auburn_researchers_hard_at_work_pioneering_energy-related_advancements_of_future.htm, title=Auburn researchers hard at work pioneering energy-related ..., description= Nov 1, 2024  ·  A team of faculty and students in mechanical engineering is working on ways to cool advanced electronics servers to help with power conservation ... )
SearchResult(url=https://www.auburn.edu/cosam/news/articles/2024/05/auburn_alumnus_bridges_auburn_university_and_the_national_renewable_energy_laboratory_in_does_energy_earthshots_initiative.htm, title=Auburn alumnus bridges Auburn University and the National ..., description= May 28, 2024  ·  Physicists of Auburn University are collaborating with this NREL led multi-institute research team on the long duration storage of renewable, sustainable ... )
SearchResult(url=https://www.auburn.edu/cosam/news/articles/2021/02/new_x-ray_diffrac

KeyboardInterrupt: 

In [ ]:
#bing scraper from chatgpt, terrible do not use this.
#https://support.microsoft.com/en-us/topic/advanced-search-options-b92e25f1-0085-4271-bdf9-14aaea720930


import requests
from bs4 import BeautifulSoup
import random
import csv
import time

# User agent rotation
user_agent_list = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:77.0) Gecko/20100101 Firefox/77.0',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.97 Safari/537.36',
    'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36',
    'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_5) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.1.1 Safari/605.1.15',
]

# Perform Bing search and parse results
def bing_search(query, max_results=10):
    headers = {'User-Agent': random.choice(user_agent_list)}
    params = {'q': query}
    response = requests.get("https://www.bing.com/search", headers=headers, params=params)

    if response.status_code != 200:
        print(f"Bing search failed: {response.status_code}")
        return []

    soup = BeautifulSoup(response.text, 'lxml')

    links = []
    for item in soup.find_all('li', class_='b_algo'):
        a_tag = item.find('a', href=True)
        if a_tag:
            links.append(a_tag['href'])
        if len(links) >= max_results:
            break

    return links

# Read from test.csv and write to output.csv
with open('test.csv', newline='') as csvfile, open('output.csv', mode='w', newline='') as outfile:
    reader = csv.reader(csvfile)
    writer = csv.writer(outfile)

    for row in reader:
        if not row: continue
        domain = row[0].strip()
        query = f"site:{domain} (battery OR energy storage)"
        print(f"\nSearching Bing: {query}")

        try:
            results = bing_search(query)
            for url in results:
                print("  Found:", url)
                writer.writerow([domain, url])
            time.sleep(random.uniform(1.5, 3.0))  # polite delay
        except Exception as e:
            print(f"Error processing {domain}: {e}")



Searching Bing: site:www.nbccosmetology.com/ (battery OR energy storage)
  Found: https://www.bing.com/ck/a?!&&p=c831f13d51ab96dddc3be95a7538c3cf885d0880b15bd274ad881bed3157fbb8JmltdHM9MTc1MTUwMDgwMA&ptn=3&ver=2&hsh=4&fclid=1539afb0-64e4-6f7a-23bf-b9ae65a86e66&u=a1aHR0cHM6Ly93d3cud2Vmb3J1bS5vcmcvc3Rvcmllcy8yMDIzLzA5L2JhdHRlcnktZW5lcmd5LXN0b3JhZ2UtcmVuZXdhYmxlLWVuZXJneS1uZXQtemVyby8&ntb=1
  Found: https://www.bing.com/ck/a?!&&p=a3c45b6267da7ed57737e842a1ff2e892a093cbd441ee6b7aeb23c15f8d88687JmltdHM9MTc1MTUwMDgwMA&ptn=3&ver=2&hsh=4&fclid=1539afb0-64e4-6f7a-23bf-b9ae65a86e66&u=a1aHR0cHM6Ly93d3cud2Vmb3J1bS5vcmcvc3Rvcmllcy8yMDIxLzA0L3JlbmV3YWJsZS1lbmVyZ3ktc3RvcmFnZS1wdW1wZWQtYmF0dGVyaWVzLXRoZXJtYWwtbWVjaGFuaWNhbC8&ntb=1
  Found: https://www.bing.com/ck/a?!&&p=5dc34dde3d17ee56d8304303ba25425092ff94b11edff0581bb45f6d7ac02825JmltdHM9MTc1MTUwMDgwMA&ptn=3&ver=2&hsh=4&fclid=1539afb0-64e4-6f7a-23bf-b9ae65a86e66&u=a1aHR0cHM6Ly93d3cud2Vmb3J1bS5vcmcvc3Rvcmllcy8yMDI0LzExL3RoZS1yb2xlLW9mLWVuZXJneS1zdG

KeyboardInterrupt: 

In [ ]:
#scrap whole page
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

url = "https://catalog.calpoly.edu/#/"

response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")

print(soup)


<!DOCTYPE html>

<html dir="ltr" lang="en" xml:lang="en">
<head>
<title>Catalog Home | Cal Poly Academic Catalog</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Cal Poly Academic Catalog" property="og:site_name"/>
<meta content="nfMKIgzWTAWZrOjhmlWm4XZZmkHpw7h80V_b5yHZYwA" name="google-site-verification"/>
<link href="/search/opensearch.xml" rel="search" title="Catalog" type="application/opensearchdescription+xml"/>
<meta content="width=device-width, initial-scale=1.0, minimum-scale=1.0" name="viewport"/>
<link href="/images/favicon.ico" rel="shortcut icon"/>
<link href="/css/reset.css" rel="stylesheet" type="text/css"/>
<link href="/css/courseleaf.css?v=1657824529000" rel="stylesheet" type="text/css"/>
<link href="/fonts/font-awesome/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<link href="/css/screen.css?v=1657824529000" media="screen" rel="stylesheet" type="text/css"/>
<link href="/css/handheld.css?v=1657824529000" media="only